In [44]:
import roboticstoolbox as rtb
import torch    

from utils import denorm_q

robot = rtb.models.DH.Cobra600() #Puma560()

model = torch.load('models/cobra600_v1.pt')
model.eval()

MLP(
  (layers): ModuleList(
    (0): Linear(in_features=4, out_features=12, bias=True)
    (1): Linear(in_features=12, out_features=12, bias=True)
    (2): Linear(in_features=12, out_features=12, bias=True)
    (3): Linear(in_features=12, out_features=12, bias=True)
    (4): Linear(in_features=12, out_features=3, bias=True)
  )
)

In [45]:
criterion = torch.nn.MSELoss()
q_test = torch.rand((3, robot.n))

real = robot.fkine(denorm_q(robot, q_test.numpy())).t
pred = model(q_test).detach()

error = criterion(pred, torch.tensor(real))

real, pred, error

(array([[ 0.46998231, -0.3274512 ,  0.18354907],
        [ 0.41221405, -0.23280998,  0.28731804],
        [ 0.3077849 ,  0.38876085,  0.21852716]]),
 tensor([[ 0.4682, -0.3307,  0.1835],
         [ 0.4122, -0.2387,  0.2879],
         [ 0.3099,  0.3850,  0.2166]]),
 tensor(7.9428e-06, dtype=torch.float64))

In [46]:
p1 = torch.tensor([1, 2, 3], dtype=torch.float)#torch.rand(3)
p2 = torch.tensor([0, 0, 0], dtype=torch.float)#torch.rand(3)

torch.sqrt(torch.sum((p1-p2)**2))/len(p1), torch.sqrt(criterion(p1,p2)/len(p1))

(tensor(1.2472), tensor(1.2472))

In [48]:
from torch.autograd.functional import jacobian

from ikine import ikine_pi_jacob

def model_fkine(q):
    return model(q).detach()

def model_jacobian(q):
    return jacobian(model, q).squeeze() #.float()

def robot_fkine(q):
    return torch.tensor(robot.fkine(q.numpy()).t, dtype=torch.float)

def robot_jacobian(q):
    return torch.tensor(robot.jacob0(q.numpy())[:3], dtype=torch.float)

q_start = denorm_q(robot, torch.rand(robot.n)).float()
p_start = robot_fkine(q_start)

q_target = denorm_q(robot, torch.rand(robot.n))
p_target = robot_fkine(q_target)

In [55]:
q_test = torch.rand(robot.n)

robot_jacobian(q_test), model_jacobian(q_test)

(tensor([[-3.9878e-01, -2.6220e-01,  1.1676e-16,  0.0000e+00],
         [ 3.7783e-01,  8.2927e-02, -3.6930e-17,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00, -1.0000e+00,  0.0000e+00]]),
 tensor([[-0.2967, -0.6580, -0.0028,  0.0040],
         [ 0.8733,  0.5351,  0.0062, -0.0039],
         [-0.0028,  0.0064, -0.2077, -0.0014]]))

In [49]:
q_inv = ikine_pi_jacob(q_start, p_target, eta=0.1,
                       fkine=robot_fkine, 
                       jacob=robot_jacobian)

p_reached = robot_fkine(q_inv)

q_inv, q_target, p_target, p_reached

(tensor([-0.7926,  0.6632,  0.0251, -0.8627]),
 tensor([-0.1868, -0.6632,  0.0251, -2.8044], dtype=torch.float64),
 tensor([ 0.5008, -0.2669,  0.3619]),
 tensor([ 0.5008, -0.2669,  0.3619]))

In [50]:
q_inv = ikine_pi_jacob(q_start, p_target, eta=0.1,
                       fkine=model_fkine, 
                       jacob=robot_jacobian)

p_reached = robot_fkine(q_inv)

q_inv, q_target, p_target, p_reached

(tensor([-62.1862,  12.2410,  -5.8807,  -0.8627]),
 tensor([-0.1868, -0.6632,  0.0251, -2.8044], dtype=torch.float64),
 tensor([ 0.5008, -0.2669,  0.3619]),
 tensor([0.5206, 0.2821, 6.2677]))

In [51]:
q_inv = ikine_pi_jacob(q_start, p_target, eta=0.1,
                       fkine=robot_fkine, 
                       jacob=model_jacobian)

p_reached = robot_fkine(q_inv)

q_inv, q_target, p_target, p_reached

(tensor([-0.1868, -0.6632,  0.0251,  0.6082]),
 tensor([-0.1868, -0.6632,  0.0251, -2.8044], dtype=torch.float64),
 tensor([ 0.5008, -0.2669,  0.3619]),
 tensor([ 0.5008, -0.2669,  0.3619]))

In [52]:
q_inv = ikine_pi_jacob(q_start, p_target, eta=0.1,
                       fkine=model_fkine, 
                       jacob=model_jacobian)

p_reached = robot_fkine(q_inv)

q_inv, q_target, p_target, p_reached

(tensor([ 0.0643,  0.6714,  0.1173, -0.8480]),
 tensor([-0.1868, -0.6632,  0.0251, -2.8044], dtype=torch.float64),
 tensor([ 0.5008, -0.2669,  0.3619]),
 tensor([0.5282, 0.2054, 0.2697]))